In [50]:
#All imports go here
from keras.models import Sequential
from keras.layers import Flatten,Dense

import cv2
import numpy as np
import pandas as pd

In [51]:
# Load data
DATA_DIR = './data/'
LOGFILE_PATH = DATA_DIR + 'driving_log.csv'
IMG_DIR = DATA_DIR + 'IMG/'

logs = pd.read_csv(LOGFILE_PATH,sep=',',header=None)


In [52]:
# Load image data.
img_paths = {}
img_paths['center'] = list(logs[0])

image_data = []

for path in img_paths['center']:
    file_name = path.split('/')[-1]
    img = cv2.imread(IMG_DIR+file_name)
    image_data.append(img)

image_data = np.array(image_data)


#Load steering angles
steering_angles  = np.array(list(logs[3]))

In [53]:
# Build and save model.
model = Sequential()
model.add(Flatten(input_shape=image_data.shape[1:]))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit(image_data, steering_angles,  validation_split=0.2, shuffle=True,verbose=2)

model.save('model.h5')

Train on 3248 samples, validate on 812 samples
Epoch 1/10
3s - loss: 11431546.9614 - val_loss: 6626.9833
Epoch 2/10
3s - loss: 6742.9121 - val_loss: 3303.8556
Epoch 3/10
3s - loss: 4709.9344 - val_loss: 3206.0857
Epoch 4/10
3s - loss: 3428.2355 - val_loss: 2175.9060
Epoch 5/10
3s - loss: 2454.0703 - val_loss: 1856.8597
Epoch 6/10
3s - loss: 2212.9226 - val_loss: 5987.9560
Epoch 7/10
3s - loss: 2683.4769 - val_loss: 1812.9627
Epoch 8/10
3s - loss: 2303.3853 - val_loss: 1758.0200
Epoch 9/10
3s - loss: 2424.4734 - val_loss: 1991.6734
Epoch 10/10
3s - loss: 2238.8377 - val_loss: 1479.3853
